In [86]:
#This is a starting point
#uncomment below to install merlin library
#!pip install merlin, nvtabular, merlin.models
#I had to downgrade keras to 12.2.0 there maybe issues regarding tensorflow in the future
#run lines below if you have also have an error ab not finding a keras package

#!pip uninstall keras
#!pip install keras==2.12.0


#These are the same imports from
#https://github.com/NVIDIA-Merlin/models/blob/main/examples/02-Merlin-Models-and-NVTabular-integration.ipynb

import os
import pandas as pd
import nvtabular as nvt
import matplotlib.pyplot as plt

from merlin.models.utils.example_utils import workflow_fit_transform
import merlin.io
import tensorflow

import merlin.models.tf as mm
from merlin.io.dataset import Dataset
from nvtabular.ops import *
from merlin.core.utils import download_file
from merlin.schema.tags import Tags

# Load and split

In [71]:
#num rows
data_size = df.shape[0]

#We're doing a 20/80 train/validate split change the .2 to change the split
train_split_ratio = int(.2 * data_size)

train_data = df[train_split_ratio:].to_parquet("train_data.parquet")
valid_data = df[:train_split_ratio].to_parquet("valid_data.parquet")

train = Dataset("train_data.parquet")
valid = Dataset("valid_data.parquet")

/Users/andrew/anaconda3/lib/python3.11/site-packages/merlin/io/dataset.py:267: UserWarning: Initializing an NVTabular Dataset in CPU mode.This is an experimental feature with extremely limited support!
  warnings.warn(
/Users/andrew/anaconda3/lib/python3.11/site-packages/merlin/io/parquet.py:343: UserWarning: Row group memory size (1648670318) (bytes) of parquet file is bigger than requested part_size (1073741824) for the NVTabular dataset.A row group memory size of 128 MB is generally recommended. You can find info on how to set the row group size of parquet files in https://nvidia-merlin.github.io/NVTabular/main/resources/troubleshooting.html#setting-the-row-group-size-for-the-parquet-files
  warnings.warn(
/Users/andrew/anaconda3/lib/python3.11/site-packages/merlin/io/dataset.py:267: UserWarning: Initializing an NVTabular Dataset in CPU mode.This is an experimental feature with extremely limited support!
  warnings.warn(


# EDA

In [43]:
df = pd.read_csv('/Users/andrew/Desktop/projects/recsys_data/2023-10-05 9_23pm.csv') 
#  ^ This needs point to your local .csv too big for github

In [46]:
#accidentally added an extra CLICKSTREAM_EVENTS_TOTAL column
df = df.drop(columns = ['CLICKSTREAM_EVENTS_TOTAL.1'])

# NVTabular

In [55]:
#This is how to iniate a dataset using NVtabular an NVIDIA 
#optimized ETL/feature engineering(i think?) library
#data = nvt.io.dataset.Dataset('data/reviews.csv')

In [54]:
# df.columns

In [48]:
categorical_user_features = ['COUNTRY', 'DERIVED_GENDER_BY_NAME', 'EVENT_NAME']
categorical_item_features = ['STYLE', 'TAXONOMY_STYLE', 'COLOR_NAME', 'PRODUCT_CLASS', 'PRODUCT_SUBCLASS', 'TEAM', 'FRANCHISE', 'PRODUCT_GROUP']

In [81]:
#Features needed to Transform still: FIRST_PURCHASE_AT, FIRST_VISIT_AT, LATEST_VISIT_AT
# LATEST_PURCHASE_AT, EVENT_TIMESTAMP

In [80]:
#DON'T FORGET TARGET ENCODING CATEGORICAL VALUES

In [96]:
user_id = ["USER_ID"] >> Categorify(dtype = "int32") >> FillMissing(0) >> TagAsUserID()
item_id = ["ITEM_ID"] >> Categorify(dtype = "int32") >> FillMissing(0)  >> TagAsItemID()
item_features = categorical_item_features >> Categorify(dtype = "int32") >> FillMissing(0)  >> TagAsItemFeatures()
user_features = categorical_user_features >> Categorify(dtype = "int32") >> FillMissing(0)  >> TagAsUserFeatures()

In [97]:
user_features

<Node TagAsUserFeatures output>

In [98]:
#Make this more succinct
CLICKSTREAM_EVENTS_TOTAL = (
    ['CLICKSTREAM_EVENTS_TOTAL']
    >>FillMissing(0)
    >>LogOp()
    >>Normalize()
    >>LambdaOp(lambda col: col.astype("float32"))
    >>TagAsUserFeatures()
)

#has the price changed over time?
PRICE_INFORMATION = (
    ['PRICE_INFORMATION']
    >>FillMissing(0)
    >>LogOp()
    >>Normalize()
    >>LambdaOp(lambda col: col.astype("float32"))
    >>TagAsItemFeatures()
)

AVG_REVIEW_SCORE = (
    ['AVG_REVIEW_SCORE']
    >>FillMissing(0)
    >>LogOp()
    >>Normalize()
    >>LambdaOp(lambda col: col.astype("float32"))
    >>TagAsItemFeatures()
)

In [99]:
outputs = (
    user_id + 
    user_features + 
    CLICKSTREAM_EVENTS_TOTAL + 
    item_id +
    item_features + 
    PRICE_INFORMATION +
    AVG_REVIEW_SCORE
)

workflow = nvt.Workflow(outputs)

In [100]:
%%time

train_data = Dataset('train_data.parquet') #Direct to your local files
valid_data = Dataset('valid_data.parquet')

workflow.fit_transform(train_data).to_parquet(
    'train_data_transformed' #Feel free to replace with your own local file path
)
workflow.transform(valid_data).to_parquet(
    'valid_data_transformed' #same here
)

/Users/andrew/anaconda3/lib/python3.11/site-packages/merlin/io/dataset.py:267: UserWarning: Initializing an NVTabular Dataset in CPU mode.This is an experimental feature with extremely limited support!
  warnings.warn(
/Users/andrew/anaconda3/lib/python3.11/site-packages/merlin/io/parquet.py:343: UserWarning: Row group memory size (1648670318) (bytes) of parquet file is bigger than requested part_size (1073741824) for the NVTabular dataset.A row group memory size of 128 MB is generally recommended. You can find info on how to set the row group size of parquet files in https://nvidia-merlin.github.io/NVTabular/main/resources/troubleshooting.html#setting-the-row-group-size-for-the-parquet-files
  warnings.warn(
/Users/andrew/anaconda3/lib/python3.11/site-packages/merlin/io/dataset.py:267: UserWarning: Initializing an NVTabular Dataset in CPU mode.This is an experimental feature with extremely limited support!
  warnings.warn(
/Users/andrew/anaconda3/lib/python3.11/site-packages/pandas/co

CPU times: user 20.8 s, sys: 3.89 s, total: 24.7 s
Wall time: 24.6 s


/Users/andrew/anaconda3/lib/python3.11/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


## Transform Timestamped data

In [101]:
train_transformed = Dataset('train_data_transformed', engine = 'parquet')
valid_transformed = Dataset('valid_data_transformed', engine = 'parquet')

/Users/andrew/anaconda3/lib/python3.11/site-packages/merlin/io/dataset.py:267: UserWarning: Initializing an NVTabular Dataset in CPU mode.This is an experimental feature with extremely limited support!
  warnings.warn(
/Users/andrew/anaconda3/lib/python3.11/site-packages/merlin/io/dataset.py:267: UserWarning: Initializing an NVTabular Dataset in CPU mode.This is an experimental feature with extremely limited support!
  warnings.warn(


In [102]:
train_transformed.head()

,USER_ID,COUNTRY,DERIVED_GENDER_BY_NAME,EVENT_NAME,CLICKSTREAM_EVENTS_TOTAL,ITEM_ID,STYLE,TAXONOMY_STYLE,COLOR_NAME,PRODUCT_CLASS,PRODUCT_SUBCLASS,TEAM,FRANCHISE,PRODUCT_GROUP,PRICE_INFORMATION,AVG_REVIEW_SCORE
0,70320,1,1,3,-1.264287,16,3,3,20,3,4,3,4,3,0.277768,0.857715
1,691,3,3,3,1.861603,48,11,8,29,3,3,4,9,4,0.411870,-0.824569
2,1471,3,3,3,1.420414,25,12,10,25,3,4,3,5,4,0.803233,0.585295
3,580,3,3,3,0.473524,7,4,4,6,3,3,4,3,3,0.544698,0.415922
4,27946,3,3,3,-0.584007,9,6,7,5,3,5,5,6,3,-0.445158,0.339810


In [108]:
model = mm.DLRMModel(
    train_transformed.schema,
    embedding_dim=64,
    bottom_block=mm.MLPBlock([128, 64]),
    top_block=mm.MLPBlock([128, 64, 32]),
    prediction_tasks=mm.BinaryOutput(
        train_transformed.schema.select_by_tag(Tags.TARGET).column_names[0]
    ),
)

model.compile(optimizer="adam")
model.fit(train_transformed, batch_size=1024)

IndexError: list index out of range

# Model

In [74]:
train

In [84]:
model = mm.DLRMModel(
    train_transformed.schema,
    embedding_dim=64,
    bottom_block=mm.MLPBlock([128, 64]),
    top_block=mm.MLPBlock([128, 64, 32]),
    prediction_tasks=mm.BinaryClassificationTask(train.schema)
)

model.compile(optimizer="adagrad", run_eagerly=False)
model.fit(train, validation_data=valid, batch_size=1024)
eval_metrics = model.evaluate(valid, batch_size=1024, return_dict=True)

ValueError: ('Binary classification task requires a column with a ', '`Tags.BINARY_CLASSIFICATION` tag.')